## Run a demo simulation

In [2]:
import sys
sys.path.append("..")   # allow Python to see the qepc package


In [3]:
import qepc
from qepc.models.nba import run_nba_game_sim

qepc.QEPC_VERSION

'v80.0-bootstrap'

In [4]:
result = run_nba_game_sim(
    home_team='Warriors',
    away_team='Celtics',
    home_strength=1.05,
    away_strength=0.98,
    pace=1.02,
    spread_line=-3.5,
    total_line=228.5,
)
result

{'league': 'NBA',
 'home_team': 'Warriors',
 'away_team': 'Celtics',
 'home_lambda': 119.95200000000001,
 'away_lambda': 111.95519999999999,
 'home_mean': 119.7604,
 'away_mean': 112.1886,
 'home_win_prob': 0.6764,
 'avg_margin': 7.5718,
 'total_mean': 231.949,
 'p_home_cover': 0.6054,
 'p_away_cover': 0.3946,
 'p_over': 0.5822,
 'p_under': 0.4178,
 'spread_line': -3.5,
 'total_line': 228.5}

# 🎛️ The Quantum Entangled Poisson Cascade  
A centralized interactive dashboard for exploring:

- Team power rankings  
- Team vs Team comparison  
- Game predictions (interactive)  
- Today's matchups  
- Visual analytics  
- (Future) Player prop explorer  

This notebook acts as a **front-end interface** to your module-based QEPC engine.


In [ ]:
import sys
sys.path.append("..")
sys.path.append("../qepc_project")

# QEPC module imports
from qepc_project.qepc.utils.loader import load_games, load_team_stats
from qepc_project.qepc.models.nba import run_nba_simulation
from qepc_project.qepc.engine.strengths import build_team_strengths
from qepc_project.qepc.engine.matchup import compute_matchup_factors

# Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

print("✅ QEPC Dashboard Ready.")


## 📂 Load Data  
Loads game results and team statistics from the `data/` folder.


In [ ]:
games = load_games('../data/Games.csv')
team_stats = load_team_stats('../data/TeamStatistics.csv')

print("Games loaded:", len(games))
print("Team stats loaded:", len(team_stats))

games.head()


## ⚙️ Build Rolling Strength Model  
This creates the offensive, defensive, and pace factors used for predictions.


In [ ]:
team_strengths = build_team_strengths(team_stats)
print("Teams with strength profiles:", len(team_strengths))
list(team_strengths.keys())[:10]


# 📊 Power Rankings  
Top teams ranked by **rolling strength score**  
(using offensive factor × defensive factor).


In [ ]:
strength_scores = {
    team: vals['off_factor'] * vals['def_factor']
    for team, vals in team_strengths.items()
}

power_rankings = (
    pd.Series(strength_scores)
    .sort_values(ascending=False)
    .rename("QEPC Power Score")
)

power_rankings.head(10)


# 🆚 Team Comparison Panel  
Select two teams to compare scoring trends and recent performance.


In [ ]:
team_list = sorted(team_stats['teamName'].unique())

team_a = widgets.Dropdown(options=team_list, description="Team A:")
team_b = widgets.Dropdown(options=team_list, description="Team B:")

display(team_a, team_b)


In [ ]:
def compare_teams(team1, team2):
    df1 = team_stats[team_stats['teamName'] == team1].tail(10)
    df2 = team_stats[team_stats['teamName'] == team2].tail(10)

    print(f"\n📘 Last 10 Games — {team1}")
    display(df1[['teamScore', 'opponentScore']].describe())

    print(f"\n📙 Last 10 Games — {team2}")
    display(df2[['teamScore', 'opponentScore']].describe())

widgets.interact(compare_teams, team1=team_a, team2=team_b);


# 🧮 Game Prediction Tool  
Pick any two teams to instantly run a QEPC prediction.


In [ ]:
home_team = widgets.Dropdown(options=team_list, description="Home:")
away_team = widgets.Dropdown(options=team_list, description="Away:")

display(home_team, away_team)


In [ ]:
def predict_game(home, away):
    if home == away:
        return "⚠️ Teams must be different."

    home_str, away_str, pace = compute_matchup_factors(home, away, team_strengths)

    sim = run_nba_simulation(
        home=home,
        away=away,
        home_strength=home_str,
        away_strength=away_str,
        pace=pace
    )

    print(f"\n🏀 QEPC Prediction — {away} @ {home}")
    print("--------------------------------------")
    print("Home Win Probability:", round(sim.home_win_prob, 3))
    print("Predicted Margin:", round(sim.avg_margin, 2))
    print("Predicted Total:", round(sim.total_mean, 1))

widgets.interact(predict_game, home=home_team, away=away_team);


# 🎯 Today's Games  
Automatically detect today's date and run QEPC predictions for all games.


In [ ]:
games['gameDate'] = pd.to_datetime(games['gameDate'])
today = pd.Timestamp.now().date()

todays_games = games[games['gameDate'].dt.date == today]

if todays_games.empty:
    print("No games today.")
else:
    for _, row in todays_games.iterrows():
        home = row['hometeamName']
        away = row['awayteamName']

        home_str, away_str, pace = compute_matchup_factors(home, away, team_strengths)
        sim = run_nba_simulation(home, away, home_str, away_str, pace)

        print(f"\n{away} @ {home}")
        print("Home win probability:", round(sim.home_win_prob, 3))
        print("Predicted margin:", round(sim.avg_margin, 2))
        print("Predicted total:", round(sim.total_mean, 1))


# 🧍‍♂️📈 Player Prop Explorer (Coming Soon)  
This panel will allow you to select a player and see predicted:  
- Points  
- Rebounds  
- Assists  
- PRA  
- 3PM  
...using QEPC's λ-based prop engine.


# 🎨 Visual Analytics  
Plots and charts to explore scoring tendencies and team efficiencies.


In [ ]:
plt.figure(figsize=(10,5))
plt.hist(team_stats['teamScore'], bins=30, edgecolor='black')
plt.title("Distribution of Team Scores")
plt.xlabel("Points Scored")
plt.ylabel("Frequency")
plt.grid(alpha=0.3)
plt.show()


# 🏁 Dashboard Ready  
Your QEPC Dashboard is now fully structured and functional.

As modules expand, we’ll add:
- Player projections  
- SGP builder  
- Odds comparison tools  
- Enhanced power ratings  
- Daily auto-updates  

Use this notebook as your main control panel for exploring QEPC outputs.
